# Aula 2 - Busca não-informada e informada

## Parte 1 - busca não-informada

Abaixo, temos a implementação do algoritmo busca em largura.

In [0]:
import collections

def bfs(graph, root): 
    visited, queue = set(), collections.deque([root])
    visited.add(root)
    while queue: 
        vertex = queue.popleft()
        print(vertex)
        for neighbour in graph[vertex]: 
            if neighbour not in visited: 
                visited.add(neighbour) 
                queue.append(neighbour) 

Grafos podem ser definidos como segue:

In [0]:
graph1 = {0: [1, 2], 1: [2], 2: [3], 3: [1, 2]} 
graph2 = {0: [1, 2], 1: [0, 3, 4], 2: [0], 3: [1], 4: [2, 3]}

Para testar o algoritmo, basta fazer o seguinte:

In [3]:
print('Ordem de visitação do BFS no graph1:')
bfs(graph1, 0)
print('Ordem de visitação do BFS no graph2:')
bfs(graph2, 0)

Ordem de visitação do BFS no graph1:
0
1
2
3
Ordem de visitação do BFS no graph2:
0
1
2
3
4


### Exercício 1

Altere o algoritmo acima para que ele realize a busca em profundidade. Sabendo que a diferença entre o BFS e o DFS diz respeito à estrutura de dados utilizada para implementar a fila, a seguinte documentação pode lhe ser útil: https://docs.python.org/2/library/collections.html#collections.deque

In [0]:
def dfs(graph, root): 
    visited, queue = set(), collections.deque([root])
    visited.add(root)
    while queue: 
        vertex = queue.pop() # now we have a LIFO queue
        print(vertex)
        for neighbour in graph[vertex]: 
            if neighbour not in visited: 
                visited.add(neighbour) 
                queue.append(neighbour) 

### Exercício 2

Crie um grafo maior (e.g., o mapa da Romênia). Em seguida, execute os dois algoritmos acima e compare a ordem com que os nós são visitados. Qual dos dois algoritmos é melhor? Por que?

In [5]:
graph3 = {
    'Arad': ['Zerind', 'Sibiu', 'Timisoara'], 
    'Zerind': ['Arad', 'Oradea'], 
    'Sibiu': ['Arad', 'Oradea', 'Rimnicu Vilcea', 'Fagaras'], 
    'Timisoara': ['Arad', 'Lugoj'],
    'Oradea': ['Zerind', 'Sibiu'],
    'Fagaras': ['Sibiu', 'Bucharest'],
    'Lugoj': ['Timisoara', 'Mehadia'],
    'Mehadia': ['Lugoj', 'Drobeta'],
    'Drobeta': ['Mehadia', 'Craiova'],
    'Craiova': ['Drobeta', 'Rimnicu Vilcea', 'Pitesti'],
    'Rimnicu Vilcea': ['Craiova', 'Sibiu', 'Pitesti'],
    'Pitesti': ['Craiova', 'Rimnicu Vilcea', 'Bucharest'],
    'Bucharest': ['Pitesti', 'Fagaras', 'Giurgiu', 'Urziceni'],
    'Giurgiu': ['Bucharest'],
    'Urziceni': ['Bucharest', 'Hirsova', 'Vaslui'],
    'Hirsova': ['Urziceni', 'Eforie'],
    'Eforie': ['Hirsova'],
    'Vaslui': ['Hirsova', 'Iasi'],
    'Iasi': ['Vaslui', 'Neamt'],
    'Neamt': ['Iasi']
    
}

print('Ordem de visitação do BFS no graph3:')
bfs(graph3, 'Arad')
print('\nOrdem de visitação do DFS no graph3:')
dfs(graph3, 'Arad')

Ordem de visitação do BFS no graph3:
Arad
Zerind
Sibiu
Timisoara
Oradea
Rimnicu Vilcea
Fagaras
Lugoj
Craiova
Pitesti
Bucharest
Mehadia
Drobeta
Giurgiu
Urziceni
Hirsova
Vaslui
Eforie
Iasi
Neamt

Ordem de visitação do DFS no graph3:
Arad
Timisoara
Lugoj
Mehadia
Drobeta
Craiova
Pitesti
Bucharest
Urziceni
Vaslui
Iasi
Neamt
Hirsova
Eforie
Giurgiu
Fagaras
Rimnicu Vilcea
Sibiu
Oradea
Zerind


No grafo da Romênia, o algoritmo de busca em profundidade se saiu melhor, tendo alcançado Bucareste em menos passos que a busca em largura. No entanto, este algoritmo não é necessariamente melhor em cada caso. Isto depende muito da ordem que os nós são explorados durante a expansão de um nó da fronteira. Além do mais, o desempenho pode variar bastante em função da instância do problema. Finalmente, nenhuma das abordagens considera o custo das arestas, sendo portanto pouco úteis em problemas de roteamento como este, onde o minimizar custo da viagem faz parte do objetivo.

## Parte 2 - busca informada

Abaixo, temos uma implementação incompleta do algoritmo A*.

In [0]:
class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def astar(maze, start, end, heuristic):
    
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    iterations = 0
    while len(open_list) > 0:
        iterations += 1

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1], iterations # Return reversed path and the number of iterations to complete the algorithm

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # up, down, right, left

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            is_in_closed = False
            for closed_child in closed_list:
                if child == closed_child:
                    is_in_closed = True
                    break
            if is_in_closed:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = heuristic(child.position[0], child.position[1], end_node.position[0], end_node.position[1])
            child.f = child.g + child.h

            # Child is already in the open list
            is_in_open = False
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    is_in_open = True
                    break
            if is_in_open:
                continue

            # Add the child to the open list
            open_list.append(child)

def f_heuristica(x1, y1, x2, y2):
    return x1-x2 + y1-y2


O código abaixo cria um ambiente do tipo labirinto.

In [0]:
maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

start = (0, 0)
end = (7, 6)

Para executar o algoritmo, basta executar o seguinte.

In [8]:
path = astar(maze, start, end, f_heuristica)
print(path)

([(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (6, 6), (7, 6)], 3132)


### Exercício 3

Implemente as seguintes heurísticas: distância Euclidiana e distância de Manhattan.

In [0]:
def h_euclidiana(x1, y1, x2, y2):
  return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

In [0]:
def h_manhattan(x1, y1, x2, y2):
  return abs(x2 - x1) + abs(y2 - y1)

### Exercício 4

Crie uma terceira heurística à sua escolha.

In [0]:
def h_fulano(x1, y1, x2, y2):
  return 0

### Exercício 5

Exercute o algoritmo A* com as três heurísticas acima e compare o resultado.

In [12]:
e, ie = astar(maze, start, end, h_euclidiana)
m, me = astar(maze, start, end, h_manhattan)
g, ge = astar(maze, start, end, h_fulano)

print('Distância Euclidiana: \n\tCaminho: %s \n\tCusto do caminho: %d\n\tIterações do algoritmo: %d' % (e, len(e), ie))
print('Distância de Manhattan: \n\tCaminho: %s \n\tCusto do caminho: %d\n\tIterações do algoritmo: %d' % (m, len(m), me))
print('Distância de fulano: \n\tCaminho: %s \n\tCusto do caminho: %d\n\tIterações do algoritmo: %d' % (g, len(g), ge))

Distância Euclidiana: 
	Caminho: [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (6, 5), (6, 6), (7, 6)] 
	Custo do caminho: 14
	Iterações do algoritmo: 878
Distância de Manhattan: 
	Caminho: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (6, 6), (7, 6)] 
	Custo do caminho: 14
	Iterações do algoritmo: 887
Distância de fulano: 
	Caminho: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (6, 6), (7, 6)] 
	Custo do caminho: 14
	Iterações do algoritmo: 1679


Conforme pode ser observado acima, as três heurísticas implementadas encontraram um caminho ótimo. De fato, todas elas são admissíveis, pois nenhuma superestima o custo do caminho. Isto aconteceu porque, nesta implementação, os caminhos possíveis são apenas `Cima`, `Baixo`, `Direita` e `Esquerda` e o custo de cada passo é sempre 1. 

No entanto, algumas destas heurísticas claramente fazem estimativas muito longe do custo real. Por exemplo, a `h_fulano` estima sempre um custo zero; consequentemente, o número de iterações necessárias para executar o algoritmo é bem maior que nos demais casos. 